In [1]:
import numpy as np
import cv2
import h5py
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Conv2D,
    Concatenate,
    MaxPool2D,
    Dropout,
    Flatten,
)

2023-03-25 23:10:55.303204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fizzer/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib
2023-03-25 23:10:55.303227: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
base_model = tf.keras.models.load_model('trained_model_0')

2023-03-25 23:10:56.473729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fizzer/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib
2023-03-25 23:10:56.473745: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-25 23:10:56.473758: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (skynet): /proc/driver/nvidia/version does not exist
2023-03-25 23:10:56.473914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
#fine_imgs1 = np.load('data/fine_imgs.npy')
#fine_vels1 = np.load('data/fine_vels.npy')
fine_imgs2 = np.load('data/fine_imgs_2.npy')
fine_vels2 = np.load('data/fine_vels_2.npy')
fine_imgs3 = np.load('data/fine_imgs_3.npy')
fine_vels3 = np.load('data/fine_vels_3.npy')
fine_imgs4 = np.load('data/fine_imgs_4.npy')
fine_vels4 = np.load('data/fine_vels_4.npy')
fine_imgs = np.concatenate([fine_imgs2, fine_imgs3, fine_imgs4])
fine_vels = np.concatenate([fine_vels2,fine_vels3,fine_vels4])
print(fine_imgs.shape)
print(fine_vels.shape)

(1597, 90, 160, 1)
(1597, 2)


In [29]:
# cv2.imshow('win', fine_imgs[1000])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [30]:
img_data = fine_imgs/255
vel_data = fine_vels

In [31]:
velocities_to_actions = {
    (0.25, 0.0): np.array([1, 0, 0]),
    (0.25, 1.0): np.array([0, 1, 0]),
    (0.25, -1.0): np.array([0, 0, 1]),
}

actions = np.array([velocities_to_actions[tuple(i)] for i in vel_data])

In [32]:
base_model.trainable=False

In [33]:
base_model.layers

In [34]:
new_model = keras.Sequential()
for layer in base_model.layers[:-1]:
    new_model.add(layer)
    
print(type(new_model))

<class 'keras.engine.sequential.Sequential'>


In [35]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 90, 160, 32)       544       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 22, 40, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 22, 40, 16)        8208      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 10, 16)        0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 5, 10, 16)         0         
                                                                 
 flatten_4 (Flatten)         (None, 800)              

In [42]:
inp = new_model.input
out = Dense(128, activation='relu')(new_model.output)
out = Dropout(0.3)(out)
out = Dense(32, activation='relu')(out)
out = Dense(3, activation='softmax')(out)
finetuner = Model(inp, out)
finetuner.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 90, 160, 1)]      0         
                                                                 
 conv2d_8 (Conv2D)           (None, 90, 160, 32)       544       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 22, 40, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 22, 40, 16)        8208      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 10, 16)        0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 5, 10, 16)         0   

In [43]:
finetuner.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [44]:
finetuner.fit(img_data, actions, batch_size=1, epochs=5, validation_split=0.2)

Epoch 1/5
1277/1277 [==============================] - 3s 2ms/step - loss: 0.4196 - accuracy: 0.8183 - val_loss: 0.1239 - val_accuracy: 0.9719
Epoch 2/5
1277/1277 [==============================] - 3s 2ms/step - loss: 0.3760 - accuracy: 0.8309 - val_loss: 0.1163 - val_accuracy: 0.9594
Epoch 3/5
1277/1277 [==============================] - 3s 2ms/step - loss: 0.3629 - accuracy: 0.8269 - val_loss: 0.1697 - val_accuracy: 0.9187
Epoch 4/5
1277/1277 [==============================] - 3s 2ms/step - loss: 0.3512 - accuracy: 0.8418 - val_loss: 0.1019 - val_accuracy: 0.9781
Epoch 5/5
1277/1277 [==============================] - 3s 2ms/step - loss: 0.3658 - accuracy: 0.8356 - val_loss: 0.1477 - val_accuracy: 0.9625


In [45]:
finetuner.save('finetuned_model')

INFO:tensorflow:Assets written to: finetuned_model/assets


INFO:tensorflow:Assets written to: finetuned_model/assets
